In [64]:
import numpy as np
import pandas as pd

In [65]:
mails_data = pd.read_csv('https://raw.githubusercontent.com/Sahanac/myfirstprogect/master/data/spam%20data')

In [66]:
mails_data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [67]:
mails_data.rename(columns = {'v1':'label','v2':'message'},inplace = True)

In [68]:
mails_data.head()

,label,message,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [69]:
mails_data.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis = 1,inplace = True)

In [70]:
mails_data.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [71]:
mails_data['length'] = mails_data['message'].apply(len)

In [72]:
mails_data.head()

,label,message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [73]:
from sklearn.preprocessing import FunctionTransformer

In [74]:
def func(x):
    def mapping(label):
        if label == 'ham':
            return 0
        else:
            return 1
    return x.map(mapping)
ft = FunctionTransformer(func,validate = False)

In [75]:
mails_data['label'] = ft.transform(mails_data.label)

In [76]:
mails_data.head()

,label,message,length
0,0,"Go until jurong point, crazy.. Available only ...",111
1,0,Ok lar... Joking wif u oni...,29
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,0,U dun say so early hor... U c already then say...,49
4,0,"Nah I don't think he goes to usf, he lives aro...",61


In [77]:
from nltk.tokenize import RegexpTokenizer

In [78]:
tokennizer = RegexpTokenizer(r'[a-zA-Z]+')

In [79]:
mails_data.message = mails_data.message.map(lambda mes: tokennizer.tokenize(mes))

In [80]:
mails_data.head()

,label,message,length
0,0,"[Go, until, jurong, point, crazy, Available, o...",111
1,0,"[Ok, lar, Joking, wif, u, oni]",29
2,1,"[Free, entry, in, a, wkly, comp, to, win, FA, ...",155
3,0,"[U, dun, say, so, early, hor, U, c, already, t...",49
4,0,"[Nah, I, don, t, think, he, goes, to, usf, he,...",61


In [81]:
from nltk.stem import SnowballStemmer

In [82]:
stemmer = SnowballStemmer(language='english')

In [83]:
mails_data.message = mails_data.message.map(lambda sen: [stemmer.stem(words) for words in sen])

In [84]:
mails_data.message = mails_data.message.map(lambda sen: ' '.join(sen))

In [85]:
mails_data.head()

,label,message,length
0,0,go until jurong point crazi avail onli in bugi...,111
1,0,ok lar joke wif u oni,29
2,1,free entri in a wkli comp to win fa cup final ...,155
3,0,u dun say so earli hor u c alreadi then say,49
4,0,nah i don t think he goe to usf he live around...,61


In [86]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [87]:
cv1 = CountVectorizer(stop_words='english')

In [88]:
tv1 = TfidfVectorizer(stop_words='english')

In [89]:
wc1 = cv1.fit_transform(mails_data.message)

In [90]:
wc2 = tv1.fit_transform(mails_data.message)

In [91]:
mails_data.label.value_counts()

0    4825
1     747
Name: label, dtype: int64

In [92]:
from sklearn.model_selection import train_test_split

In [93]:
trainx,testx,trainy,testy,trainz,testz = train_test_split(wc1,wc2,mails_data.label)

In [100]:
from sklearn.linear_model import LogisticRegression
model1 = LogisticRegression()
model1_1 = LogisticRegression()
model1_2 = LogisticRegression()
model1_1.fit(trainx,trainz)
print('accuracy of model1_1 is:',model1_1.score(testx,testz)*100)
model1_2.fit(trainy,trainz)
print('accuracy of model1_2 is:',model1_2.score(testy,testz)*100)




accuracy of model1_1 is: 97.48743718592965
accuracy of model1_2 is: 95.04666188083274


/Users/sahana/anaconda3/envs/machinelearning/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [101]:
from sklearn.naive_bayes import MultinomialNB
model2 = MultinomialNB(class_prior=[0.8,0.2])
model2_1 = MultinomialNB(class_prior=[0.8,0.2])
model2_2 = MultinomialNB(class_prior=[0.8,0.2])
model2_1.fit(trainx,trainz)
print('accuracy of model2_1 is:',model2_1.score(testx,testz)*100)
model2_2.fit(trainy,trainz)
print('accuracy of model2_2 is:',model2_2.score(testy,testz)*100)


accuracy of model2_1 is: 97.48743718592965
accuracy of model2_2 is: 96.8413496051687


In [102]:
from sklearn.ensemble import RandomForestClassifier
model3 = RandomForestClassifier()
model3_1 = RandomForestClassifier()
model3_2 = RandomForestClassifier()
model3_1.fit(trainx,trainz)
print('accuracy of model3_1 is:',model3_1.score(testx,testz)*100)
model3_2.fit(trainy,trainz)
print('accuracy of model3_2 is:',model3_2.score(testy,testz)*100)


/Users/sahana/anaconda3/envs/machinelearning/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


accuracy of model3_1 is: 96.8413496051687


/Users/sahana/anaconda3/envs/machinelearning/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


accuracy of model3_2 is: 96.19526202440775


In [103]:
from sklearn.neighbors import KNeighborsClassifier,RadiusNeighborsClassifier
model4 = KNeighborsClassifier()
model4_1 = KNeighborsClassifier()
model4_2 = KNeighborsClassifier()
model4_1.fit(trainx,trainz)
print('accuracy of model4_1 is:',model4_1.score(testx,testz)*100)
model4_2.fit(trainy,trainz)
print('accuracy of model4_2 is:',model4_2.score(testy,testz)*100)

model41 = RadiusNeighborsClassifier()
model41_1 = RadiusNeighborsClassifier(radius=20)
model41_2 = RadiusNeighborsClassifier(radius=20)
model41_1.fit(trainx,trainz)
print('accuracy of model41_1 is:',model41_1.score(testx,testz)*100)
model41_2.fit(trainy,trainz)
print('accuracy of model41_2 is:',model41_2.score(testy,testz)*100)


accuracy of model4_1 is: 91.60086145010769
accuracy of model4_2 is: 90.38047379755922
accuracy of model41_1 is: 85.78607322325915
accuracy of model41_2 is: 85.78607322325915


In [104]:
from sklearn.svm import SVC
model5 = SVC(kernel='sigmoid',gamma=1.0)
model5_1 = SVC(kernel='sigmoid',gamma=1.0)
model5_2 = SVC(kernel='sigmoid',gamma=1.0)
model5_1.fit(trainx,trainz)
print('accuracy of model5_1 is:',model5_1.score(testx,testz)*100)
model5_2.fit(trainy,trainz)
print('accuracy of model5_2 is:',model5_2.score(testy,testz)*100)

accuracy of model5_1 is: 93.03661162957646
accuracy of model5_2 is: 97.20028715003589


# spam detection using pipeline

In [105]:
import numpy as np
import pandas as pd

In [106]:
mails_data1 = pd.read_csv('https://raw.githubusercontent.com/Sahanac/myfirstprogect/master/data/spam%20data')

In [107]:
mails_data1.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [108]:
mails_data1.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis = 1,inplace = True)

In [109]:
mails_data1.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [110]:
mails_data1.rename(columns = {'v1':'label','v2':'message'},inplace = True)

In [111]:
mails_data1.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [112]:
mails_data1['length'] = mails_data1['message'].apply(len)

In [113]:
mails_data1.head()

,label,message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [114]:
from sklearn.preprocessing import FunctionTransformer

In [115]:
def func(y):
    def mapping(label):
        if label == 'spam':
            return 1
        else:
            return 0
    return y.map(mapping)
ft = FunctionTransformer(func,validate = False)

In [116]:
mails_data1['label'] = ft.transform(mails_data1.label)

In [117]:
mails_data1.head()

,label,message,length
0,0,"Go until jurong point, crazy.. Available only ...",111
1,0,Ok lar... Joking wif u oni...,29
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,0,U dun say so early hor... U c already then say...,49
4,0,"Nah I don't think he goes to usf, he lives aro...",61


In [118]:
from nltk.tokenize import RegexpTokenizer

In [119]:
tokenizer = RegexpTokenizer(r'[a-zA-z]+')

In [120]:
mails_data1.message = mails_data1.message.map(lambda mes: tokenizer.tokenize(mes) )

In [121]:
mails_data1.head()

,label,message,length
0,0,"[Go, until, jurong, point, crazy, Available, o...",111
1,0,"[Ok, lar, Joking, wif, u, oni]",29
2,1,"[Free, entry, in, a, wkly, comp, to, win, FA, ...",155
3,0,"[U, dun, say, so, early, hor, U, c, already, t...",49
4,0,"[Nah, I, don, t, think, he, goes, to, usf, he,...",61


In [122]:
from nltk.stem import SnowballStemmer

In [123]:
stemmer = SnowballStemmer(language='english')

In [124]:
mails_data1.message = mails_data1.message.map(lambda sen: [stemmer.stem(words) for words in sen])

In [125]:
mails_data1.head()

,label,message,length
0,0,"[go, until, jurong, point, crazi, avail, onli,...",111
1,0,"[ok, lar, joke, wif, u, oni]",29
2,1,"[free, entri, in, a, wkli, comp, to, win, fa, ...",155
3,0,"[u, dun, say, so, earli, hor, u, c, alreadi, t...",49
4,0,"[nah, i, don, t, think, he, goe, to, usf, he, ...",61


In [126]:
mails_data1.message = mails_data1.message.map(lambda sen:' '.join(sen))

In [127]:
mails_data1.head()

,label,message,length
0,0,go until jurong point crazi avail onli in bugi...,111
1,0,ok lar joke wif u oni,29
2,1,free entri in a wkli comp to win fa cup final ...,155
3,0,u dun say so earli hor u c alreadi then say,49
4,0,nah i don t think he goe to usf he live around...,61


In [128]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,TfidfTransformer

In [129]:
cv1 = CountVectorizer()

In [130]:
tv1 = TfidfVectorizer()

In [131]:
wc1 = cv1.fit_transform(mails_data1.message)

In [132]:
wc1

<5572x6337 sparse matrix of type '<class 'numpy.int64'>'
	with 71499 stored elements in Compressed Sparse Row format>

In [133]:
wc2 = tv1.fit_transform(mails_data1.message)

In [134]:
wc2

<5572x6337 sparse matrix of type '<class 'numpy.float64'>'
	with 71499 stored elements in Compressed Sparse Row format>

In [135]:
from sklearn.model_selection import train_test_split

In [136]:
trainx,testx,trainy,testy = train_test_split(mails_data1.message,mails_data1.label)

In [137]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier,RadiusNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

In [138]:
pipelines = []
for models in [LogisticRegression(),MultinomialNB(),KNeighborsClassifier(),RadiusNeighborsClassifier(radius=30),SVC(kernel='sigmoid',gamma=1.0),RandomForestClassifier()]:
    pipeline=make_pipeline(CountVectorizer(),TfidfTransformer(),models)
    pipelines.append(pipeline)
    
    
    

In [139]:
mails_data1.message.head()

0    go until jurong point crazi avail onli in bugi...
1                                ok lar joke wif u oni
2    free entri in a wkli comp to win fa cup final ...
3          u dun say so earli hor u c alreadi then say
4    nah i don t think he goe to usf he live around...
Name: message, dtype: object

In [140]:
mails_data1.label.head()

0    0
1    0
2    1
3    0
4    0
Name: label, dtype: int64

In [141]:
for pipeline in pipelines:
    print(pipeline.steps)
    pipeline.fit(trainx,trainy)

[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('tfidftransformer', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)), ('logisticregression', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]
[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features

/Users/sahana/anaconda3/envs/machinelearning/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('tfidftransformer', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)), ('radiusneighborsclassifier', RadiusNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
             metric_params=None, n_jobs=None, outlier_label=None, p=2,
             radius=30, weights='uniform'))]
[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, s

/Users/sahana/anaconda3/envs/machinelearning/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [142]:
for pipeline in pipelines:
    print(pipeline.score(testx,testy)*100)

96.8413496051687
95.5491744436468
90.95477386934674
86.21679827709978
98.1335247666906
96.76956209619526


In [143]:
pipelines1 = []
for models1 in [LogisticRegression(),MultinomialNB(),KNeighborsClassifier(),RadiusNeighborsClassifier(radius=30),SVC(kernel='sigmoid',gamma=1.0),RandomForestClassifier()]:
    pipeline1 = make_pipeline(TfidfVectorizer(),models1)
    pipelines1.append(pipeline1)
    
    

In [144]:
for pipeline1 in pipelines1:
    pipeline1.fit(trainx,trainy)

/Users/sahana/anaconda3/envs/machinelearning/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/sahana/anaconda3/envs/machinelearning/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [145]:
for pipeline1 in pipelines1:
    print(pipeline1.score(testx,testy)*100)

96.8413496051687
95.5491744436468
90.95477386934674
86.21679827709978
98.1335247666906
96.8413496051687
